In [25]:
import os
import json
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.schema import Document

In [26]:
with open("../data/insurance_policies.json", "r") as f:
    data = json.load(f)

In [27]:

docs = [
    Document(
        page_content=(
            f"{d['title']}\n"
            f"Type: {d['type']}\n"
            f"Max Benefit: {d['max_benefit']}\n"
            f"Coverage: {', '.join(d['coverage'])}\n"
            f"Conditions: {', '.join(d['conditions'])}\n"
        ),
        metadata={"source": d["id"]}
    )
    for d in data
]

In [28]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    persist_directory="../chroma_store"
)
vectorstore.persist()

In [29]:
os.environ["GOOGLE_API_KEY"] = "api key"

In [30]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    chain_type="stuff"
)


In [39]:
query = "What is the eligibility for international travel insurance?"
response = qa_chain.run(query)

In [41]:
print(response)

The international travel insurance is available to travelers aged 0 to 70 undertaking international journeys. All applicants must submit valid identity and income proofs. Medical screening may be required based on age and health history.


In [42]:
print(qa_chain.run("maximum value i can avail from travel insurance"))

The maximum benefit you can avail from the listed travel insurance plans (Plan #1016, Plan #1022, and Plan #1028) is ₹15,00,000.


In [43]:
print(qa_chain.run("give me all the details about the health insurance"))

Comprehensive Health Insurance (Policy Number: 1075):
*   **Coverage Overview:** Hospitalization, outpatient care, maternity, and emergency services. Designed to offer financial protection against unforeseen events and liabilities.
*   **Eligibility Criteria:** Individuals aged 18 to 65 and dependent children up to age 25. All applicants must submit valid identity and income proofs. Medical screening may be required based on age and health history.
*   **Policy Terms:** Coverage is subject to applicable sub-limits and co-payments as detailed in the policy schedule. Pre-existing diseases are covered after a waiting period of 36 months from the policy start date.
*   **Renewal & Cancellation:** The policy must be renewed annually before the due date. The insurer reserves the right to cancel the policy in case of non-payment of premium, fraud, or violation of policy terms.
*   **Governing Body:** Governed by the Insurance Regulatory and Development Authority of India (IRDAI) guidelines.



In [49]:
print(qa_chain.run("What is the maximum benefit of health insurance policy"))

The maximum benefit for the health insurance policies listed (CareSecure Health Plus - Plan #1035, CareSecure Health Plus - Plan #1033, and CareSecure Health Plus - Plan #1041) is ₹10,00,000.
